In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl

from pathlib import Path
import os.path
import sys

Using TensorFlow backend.


In [2]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [3]:
participants = list(range(1, 21))
smartphones = ["N5X"]
# TODO remove
participants = list(range(1, 4))
window_size = 20

In [4]:
segments = list()
labels = list()
groups = list()

for pid in participants:
    for smartphone in smartphones:
        data = pkl.load(open(str(Path.home()) + "/data/pickles/fapra_imu-processed-" +  str(pid) + "-" + smartphone + ".pkl", "rb"))
        points = data[0]
        intervals = data[1]
        # split the points to according group
        cross = points[:,:2]
        pressed = points[:,2:]
        
        # iterate over intervals
        for i, c in zip(intervals, pressed):
            samples_per_interval = len(i[0]) - (window_size - 1)
            # create array of same label
            tmp_labels = [c] * samples_per_interval
            tmp_labels = np.array(tmp_labels)
            labels.append(tmp_labels)
            tmp_groups = [pid] * samples_per_interval
            groups = groups + tmp_groups
            for k in range(len(i.T[(window_size - 1):])):
                chunk = np.array(i.T[k:(k + window_size)])
                segments.append(chunk)

# make numpy arrays of it
labels = np.concatenate(np.array(labels))
segments = np.array(segments)
groups = np.array(groups, dtype=np.int8)

In [5]:
print(labels.shape)
print(segments.shape)
print(groups.shape)

(1021339, 2)
(1021339, 20, 18)
(1021339,)


In [6]:
## defining parameters for the input and network layers
## we are treating each segmeent or chunk as a 2D image (90 X 18)
numOfRows = segments.shape[1]
numOfColumns = segments.shape[2]

## reshaping the data for network input
reshapedSegments = segments.reshape(segments.shape[0], numOfRows, numOfColumns,1)

## Split Training- & Test-Data

In [7]:
logo = LeaveOneGroupOut()
logo.get_n_splits(segments, labels, groups)

3

In [8]:
## ConvLSTM net hyperparameters
numChannels = 1
numFilters = 128 # number of filters in Conv2D layer
# kernal size of the Conv2D layer
kernalSize1 = 2
# max pooling window size
poolingWindowSz = 2
# number of filters in fully connected layers
numNueronsFCL1 = 128
numNueronsFCL2 = 128
# number of epochs
Epochs = 10
# batchsize
batchSize = 10
# number of total clases
numClasses = labels.shape[1]
# dropout ratio for dropout layer
dropOutRatio = 0.2

In [ ]:
tf.get_default_graph()
for index, (train_index, test_index) in enumerate(logo.split(reshapedSegments, labels, groups)):

    # print('TRAIN:', train_index, 'TEST:', test_index)
    trainX, testX = reshapedSegments[train_index], reshapedSegments[test_index]
    trainY, testY = labels[train_index], labels[test_index]
    
    model = None
    model = Sequential()
    # adding the first convLSTM layer with 32 filters and 5 by 5 kernal size, using the rectifier as the activation function
    model.add(ConvLSTM2D(numFilters, (kernalSize1,kernalSize1),input_shape=(None, numOfRows, numOfColumns, 1),activation='relu', padding='same',return_sequences=True))

    ## adding a maxpooling layer
    model.add(TimeDistributed(MaxPooling2D(pool_size=(poolingWindowSz,poolingWindowSz),padding='valid')))

    ## adding a dropout layer for the regularization and avoiding over fitting
    model.add(Dropout(dropOutRatio))

    ## flattening the output in order to apple dense layer
    model.add(TimeDistributed(Flatten()))

    ## adding first fully connected layer with 256 outputs
    model.add(Dense(numNueronsFCL1, activation='relu'))

    ## adding second fully connected layer 128 outputs
    model.add(Dense(numNueronsFCL2, activation='relu'))

    ## flattening the output in order to apply the fully connected layer
    model.add(TimeDistributed(Flatten()))

    ## adding softmax layer for the classification
    model.add(Dense(numClasses, activation='softmax'))

    ## Compiling the model to generate a model
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
    def euc(y_true, y_pred):
        return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))
    
    model.compile(loss=rmse, optimizer=sgd, metrics=[rmse,euc])

    model.summary()
    
    history = model.fit(np.expand_dims(trainX,1),np.expand_dims(trainY,1), 
                        validation_data=(np.expand_dims(testX,1),np.expand_dims(testY,1)),
                        epochs=9999,batch_size=100,verbose=1)

    score = model.evaluate(np.expand_dims(testX,1),np.expand_dims(testY,1),verbose=2)
    print('%s: %.2f' % (model.metrics_names[1], score[1]))
    print('Baseline ConvLSTM Error: %.2f' %(1-score[1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 20, 18, 128) 264704    
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 10, 9, 128)  0         
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 10, 9, 128)  0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 11520)       0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 128)         1474688   
_________________________________________________________________
dense_2 (Dense)              (None, None, 128)         16512     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 128)         0         
__________

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



679200/679200 [==============================] - 345s 508us/step - loss: 0.2283 - rmse: 0.2283 - euc: 0.3229 - val_loss: 0.2816 - val_rmse: 0.2816 - val_euc: 0.3983
Epoch 9/9999
 25700/679200 [>.............................] - ETA: 4:41 - loss: 0.2268 - rmse: 0.2268 - euc: 0.3208

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



679200/679200 [==============================] - 345s 508us/step - loss: 0.2271 - rmse: 0.2271 - euc: 0.3211 - val_loss: 0.2840 - val_rmse: 0.2840 - val_euc: 0.4017
Epoch 10/9999
377600/679200 [===============>..............] - ETA: 2:09 - loss: 0.2261 - rmse: 0.2261 - euc: 0.3197

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



655900/679200 [===========================>..] - ETA: 10s - loss: 0.2247 - rmse: 0.2247 - euc: 0.3178

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



679200/679200 [==============================] - 345s 508us/step - loss: 0.2236 - rmse: 0.2236 - euc: 0.3162 - val_loss: 0.2818 - val_rmse: 0.2818 - val_euc: 0.3986
Epoch 13/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.2226 - rmse: 0.2226 - euc: 0.3148 - val_loss: 0.2843 - val_rmse: 0.2843 - val_euc: 0.4020
Epoch 14/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.2215 - rmse: 0.2215 - euc: 0.3133 - val_loss: 0.2860 - val_rmse: 0.2860 - val_euc: 0.4044
Epoch 15/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.2205 - rmse: 0.2205 - euc: 0.3118 - val_loss: 0.2831 - val_rmse: 0.2831 - val_euc: 0.4003
Epoch 16/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.2196 - rmse: 0.2196 - euc: 0.3105 - val_loss: 0.2799 - val_rmse: 0.2799 - val_euc: 0.3958
Epoch 17/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.2186 - rmse: 0.2186 - euc: 0.3

Epoch 58/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.1979 - rmse: 0.1979 - euc: 0.2799 - val_loss: 0.2956 - val_rmse: 0.2956 - val_euc: 0.4181
Epoch 59/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.1977 - rmse: 0.1977 - euc: 0.2796 - val_loss: 0.2916 - val_rmse: 0.2916 - val_euc: 0.4124
Epoch 60/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.1974 - rmse: 0.1974 - euc: 0.2791 - val_loss: 0.3018 - val_rmse: 0.3018 - val_euc: 0.4268
Epoch 61/9999
679200/679200 [==============================] - 345s 508us/step - loss: 0.1972 - rmse: 0.1972 - euc: 0.2789 - val_loss: 0.2974 - val_rmse: 0.2974 - val_euc: 0.4205
Epoch 62/9999
679100/679200 [============================>.] - ETA: 0s - loss: 0.1969 - rmse: 0.1969 - euc: 0.2784